In [7]:
import os 
import json
from PIL import Image

def get_image_metadata(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        format = img.format
        exif_data = img._getexif()
        orientation = exif_data.get(0x0112) if 0x0112 in exif_data else 'N/A'
        creation_date = exif_data.get(0x0132, 'N/A') if 0x0132 in exif_data else 'N/A'
        camera_model = exif_data.get(0x0110, 'N/A') if 0x0110 in exif_data else 'N/A'

        metadata = {
            'filename': os.path.basename(image_path),
            'size': os.path.getsize(image_path),
            'format': format,
            'orientation': orientation,
            'width': width,
            'height': height,
            'creation_date': creation_date,
            'camera_model': camera_model
            # Ajoutez d'autres métadonnées si nécessaire
        }
        
        return metadata
    
dossier_images = "images"
metadata_images = []
for filename in os.listdir(dossier_images):
    if filename.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(dossier_images, filename)
        metadata = get_image_metadata(image_path)
        metadata_images.append(metadata)

json_path = "metadata_images.json"
with open(json_path, 'w') as json_file:
    json.dump(metadata_images, json_file, indent=4)
print("ok")

TypeError: argument of type 'NoneType' is not iterable